# Label / Connected Components

Use the `label` function from `elf.parallel` to label images and volumes by applying connected components.

In [ ]:
import time

# import napari for data visualisation
import napari
import numpy as np

# import the relevant elf functionality
from elf.parallel.label import label
from elf.evaluation import rand_index

# use skimage.label to verify the elf implementation
from skimage.measure import label as label_reference

# we use binary blobs from skimage as reference data
from skimage.data import binary_blobs

## 2D Data

In [ ]:
# we first apply the connected components to a 2d image to check out the functionality
blobs = binary_blobs(length=1024, n_dim=2, volume_fraction=0.25)

In [ ]:
# compute the reference labeling with skimage
# (note that elf currently only supports nearest neighbor connectivity, so we set the connectivity to 1)
labeled_ref = label_reference(blobs, connectivity=1)

In [ ]:
# compute the labeling with elf.parallel.

# We need to allocate the output first for the elf function. 
# It is implemented like this because the function can be applied to large data (e.g. stored as hdf5 or zarr data)
# without loading it into memory. So the output also needs to be allocated beforehand to support these use-cases.
labeled_elf = np.zeros(blobs.shape, dtype="uint32")

# Set the block shape: the labeling is first computed for blocks of this size independently and in parallel.
# The labeling result is then merged across the block boundaries.
block_shape = (256, 256)
labeled_elf = label(blobs, labeled_elf, block_shape=block_shape)

In [ ]:
# check that the results agree via the rand index
_, ri = rand_index(labeled_elf, labeled_ref)
print("The rand index with the reference implementation is", ri)
print("(1. means complete agreement.)")

In [ ]:
# visualize the results
viewer = napari.Viewer()
viewer.add_image(blobs)
viewer.add_labels(labeled_ref)
viewer.add_labels(labeled_elf)
napari.run()

## 3D Data

In [ ]:
from tqdm import trange
# now we apply the connected components to a larger 3d volume to also compare the runtimes
blobs = np.zeros((1024, 1024, 1024), dtype="bool")  # computing the blobs in 3d takes too long!
for z in trange(blobs.shape[0]):
    blobs[z] = binary_blobs(length=1024, n_dim=2, volume_fraction=0.2)

In [ ]:
# compute the labeling with elf. parallel
labeled_elf = np.zeros(blobs.shape, dtype="uint32")
block_shape = (256, 256, 256)

# this time we set the verbose flag to get progress bars when we apply the labeling
t0 = time.time()
labeled_elf = label(blobs, labeled_elf, block_shape=block_shape, verbose=True)
print("Computing the labeling with elf.parallel took", time.time() - t0, "s")

In [ ]:
t0 = time.time()
labeled_ref = label_reference(blobs, connectivity=1)
print("Computing the labeling with skimage took", time.time() - t0, "s")

In [ ]:
# check that the results agree via the rand index
_, ri = rand_index(labeled_elf, labeled_ref)
print("The rand index with the reference implementation is", ri)
print("(1. means complete agreement.)")